# LangGraph와 AgentCore Memory Hooks (장기 메모리)

## 소개

이 노트북은 LangGraph 프레임워크를 사용하여 Amazon Bedrock AgentCore Memory 기능을 대화형 AI agent와 통합하는 방법을 보여줍니다. 여러 대화 세션에 걸친 **장기 메모리** 보존에 중점을 두며, agent가 과거 상호작용에서 사용자 선호도, 식이 제한 사항 및 컨텍스트 정보를 추출하고 기억할 수 있도록 합니다.

## 튜토리얼 세부 정보

| 정보                | 세부사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 장기 대화형                                                                       |
| Agent 사용 사례     | 영양 어시스턴트                                                                   |
| Agentic Framework   | LangGraph                                                                        |
| LLM model           | Anthropic Claude Haiku 4.5                                                     |
| 튜토리얼 구성 요소  | AgentCore 장기 메모리, 커스텀 메모리 전략, Pre/Post Model Hooks                  |
| 예제 복잡도         | 중급                                                                              |

다음을 배우게 됩니다:
- UserPreference 커스텀 오버라이드 전략을 사용한 AgentCore Memory 생성
- 자동 메모리 저장 및 검색을 위한 pre/post model hooks 구현
- 세션 간 사용자 선호도를 기억하는 영양 어시스턴트 구축
- 관련 사용자 컨텍스트를 검색하기 위한 시맨틱 검색 사용
- 커스텀 메모리 추출 및 통합 프롬프트 구성

### 시나리오 컨텍스트

이 예제에서는 식이 제한 사항, 좋아하는 음식, 요리 선호도 및 건강 목표를 포함하여 여러 대화에 걸쳐 사용자 컨텍스트를 기억할 수 있는 **영양 어시스턴트**를 만듭니다. Agent는 대화에서 사용자 선호도를 자동으로 추출하고 저장한 다음, 향후 상호작용을 위해 관련 컨텍스트를 검색하여 개인화된 영양 조언을 제공합니다.

## 아키텍처

<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 사전 요구 사항

- Python 3.10+
- 적절한 권한이 있는 AWS 계정
- AgentCore Memory에 대한 적절한 권한이 있는 AWS IAM 역할
- Amazon Bedrock models에 대한 액세스

환경 설정을 시작해 봅시다!

In [ ]:
# Install necessary libraries from https://github.com/langchain-ai/langchain-aws
%pip install -qr requirements.txt

In [ ]:
import os
import logging

# Import LangGraph and LangChain components
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import RunnableConfig
from langgraph.store.base import BaseStore
import uuid


region = os.getenv("AWS_REGION", "us-east-1")  # AWS 리전 설정 (기본값: us-east-1)
logging.getLogger("math-agent").setLevel(logging.DEBUG)

In [ ]:
# Import the AgentCoreMemoryStore that we will use as a store
from langgraph_checkpoint_aws import AgentCoreMemoryStore

# For this example, we will just use an InMemorySaver to save context.
# In production, we highly recommend the AgentCoreMemorySaver as a checkpointer which works seamlessly alongside the memory store
# from langgraph_checkpoint_aws import AgentCoreMemorySaver
from langgraph.checkpoint.memory import InMemorySaver  # 메모리 내 체크포인트 저장용
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

from custom_memory_prompts import consolidation_prompt, extraction_prompt  # 커스텀 메모리 프롬프트

In [ ]:
memory_name = "NutritionAssistant"
client = MemoryClient(region_name=region)
MODEL_ID = "global.anthropic.claude-haiku-4-5-20251001-v1:0"

# AgentCore Memory 생성 또는 가져오기
memory = client.create_or_get_memory(
    name=memory_name,
    description="Nutrition assistant",
    memory_execution_role_arn="arn:aws:iam::YOUR_ACCOUNT:role/YOUR_ROLE",  # Please provide a role with a valid trust policy
    strategies=[
        {
            StrategyType.CUSTOM.value: {
                "name": "NutritionPreferences",
                "description": "Captures customer food preferences and behavior",
                "namespaces": ["/{actorId}/preferences/"],  # 사용자별 메모리 namespace
                "configuration": {
                    "userPreferenceOverride": {
                        "extraction": {  # 대화에서 선호도 추출 설정
                            "appendToPrompt": extraction_prompt,
                            "modelId": MODEL_ID,
                        },
                        "consolidation": {  # 추출된 정보 통합 설정
                            "appendToPrompt": consolidation_prompt,
                            "modelId": MODEL_ID,
                        },
                    }
                },
            }
        },
    ],
)
memory_id = memory["id"]

### Memory 구성 개요

AgentCore Memory 설정에는 다음이 포함됩니다:

- **Custom Strategy**: 대화에서 영양 선호도 추출
- **Namespaces**: 사용자별로 메모리 구성 (`{actorId}/preferences/`)
- **Custom Prompts**: 음식 선호도를 위한 특화된 추출 및 통합 로직
- **Model Integration**: 메모리 처리를 위해 Claude 3.7 Sonnet 사용

메모리 시스템은 임시적이거나 관련 없는 정보를 필터링하면서 지속적인 사용자 선호도를 추출하기 위해 대화를 자동으로 처리합니다.

## 단계 3: Memory Store 및 LLM 초기화

이제 AgentCore Memory Store와 언어 model을 초기화하겠습니다.

In [ ]:
# Initialize the store to enable long term memory saving and retrieval
store = AgentCoreMemoryStore(memory_id=memory_id, region_name=region)

# Initialize Bedrock LLM
llm = init_chat_model(MODEL_ID, model_provider="bedrock_converse", region_name=region)

## 단계 4: Memory Hooks 구현

메모리 저장 및 검색을 자동으로 처리하기 위해 pre 및 post model hooks를 생성합니다:

- **Pre-model hook**: 관련 사용자 선호도를 검색하고(시맨틱 검색 기반) LLM 호출 전에 컨텍스트를 추가합니다
- **Post-model hook**: 장기 메모리 추출을 위해 대화 메시지를 저장합니다

### Memory 처리 작동 방식

1. 메시지가 actor_id 및 session_id와 함께 AgentCore Memory에 저장됩니다
2. 커스텀 전략이 대화를 처리하여 영양 선호도를 추출합니다
3. 추출된 선호도는 `{actorId}/preferences/` namespace에 저장됩니다
4. 향후 대화에서 컨텍스트를 위해 관련 선호도를 검색하고 가져올 수 있습니다

**참고**: LangChain 메시지 타입은 store에 의해 내부적으로 AgentCore Memory 메시지 타입으로 변환되어 장기 메모리로 적절하게 추출될 수 있습니다.

In [ ]:
def pre_model_hook(state, config: RunnableConfig, *, store: BaseStore):
    """Hook that runs pre-LLM invocation to save the latest human message"""
    actor_id = config["configurable"]["actor_id"]
    thread_id = config["configurable"]["thread_id"]
    # Saving the message to the actor and session combination that we get at runtime
    namespace = (actor_id, thread_id)

    messages = state.get("messages", [])
    # Save the last human message we see before LLM invocation
    for msg in reversed(messages):  # 역순으로 순회하여 최신 메시지 찾기
        if isinstance(msg, HumanMessage):
            store.put(namespace, str(uuid.uuid4()), {"message": msg})  # 사용자 메시지 저장
            break
    # Retrieve user preferences based on the last message and append to state
    user_preferences_namespace = (actor_id, "preferences/")
    preferences = store.search(user_preferences_namespace, query=msg.content, limit=5)  # 시맨틱 검색으로 관련 선호도 조회

    # Construct another AI message to add context before the current message
    if preferences:
        context_items = [pref.value for pref in preferences]
        context_message = AIMessage(  # 검색된 선호도를 컨텍스트로 추가
            content=f"[User Context: {', '.join(str(item) for item in context_items)}]"
        )
        # Insert the context message before the last human message
        return {"messages": messages[:-1] + [context_message, messages[-1]]}

    return {"llm_input_messages": messages}


def post_model_hook(state, config: RunnableConfig, *, store: BaseStore):
    """Hook that runs post-LLM invocation to save the latest human message"""
    actor_id = config["configurable"]["actor_id"]
    thread_id = config["configurable"]["thread_id"]

    # Saving the message to the actor and session combination that we get at runtime
    namespace = (actor_id, thread_id)

    messages = state.get("messages", [])
    # Save the LLMs response to AgentCore Memory
    for msg in reversed(messages):  # 역순으로 순회하여 최신 메시지 찾기
        if isinstance(msg, AIMessage):
            store.put(namespace, str(uuid.uuid4()), {"message": msg})  # AI 응답 저장
            break

    return {"messages": messages}

## 단계 5: LangGraph Agent 생성

이제 메모리 hooks가 통합된 LangGraph의 `create_react_agent`를 사용하여 영양 어시스턴트 agent를 생성합니다. tool 노드에는 장기 메모리 검색 tool만 포함되며 pre 및 post model hooks는 인수로 지정됩니다.

**참고**: 커스텀 agent 구현의 경우 Store와 tools는 이 패턴을 따라 모든 워크플로우에 필요에 따라 구성할 수 있습니다. Pre/post model hooks를 사용하거나, 전체 대화를 마지막에 저장하는 등의 방식이 가능합니다.

In [ ]:
graph = create_react_agent(
    llm,
    store=store,
    tools=[],  # No additional tools needed for this example
    checkpointer=InMemorySaver(),  # For conversation state management
    pre_model_hook=pre_model_hook,  # Retrieves user preferences before LLM call
    post_model_hook=post_model_hook,  # Saves conversation after LLM response
)

## 단계 6: Agent 런타임 구성

사용자 및 세션에 대한 고유 식별자로 agent를 구성해야 합니다. 이러한 ID는 메모리 구성 및 검색에 중요합니다.

### Graph Invoke Input
인수 `inputs`로 최신 사용자 메시지만 전달하면 됩니다. 다른 상태 변수도 포함할 수 있지만 간단한 `create_react_agent`의 경우 메시지만 필요합니다.

### LangGraph RuntimeConfig
LangGraph에서 config는 호출 시점에 필요한 속성(예: 사용자 ID 또는 세션 ID)을 포함하는 `RuntimeConfig`입니다. `AgentCoreMemorySaver`의 경우 config에 `thread_id`와 `actor_id`를 설정해야 합니다. 예를 들어, AgentCore 호출 엔드포인트는 호출자의 identity 또는 사용자 ID를 기반으로 이를 할당할 수 있습니다. [여기에서 추가 문서](https://langchain-ai.github.io/langgraphjs/how-tos/configuration/)를 읽을 수 있습니다.



In [ ]:
actor_id = "user-1"
config = {
    "configurable": {
        "thread_id": "session-1",  # REQUIRED: This maps to Bedrock AgentCore session_id under the hood
        "actor_id": actor_id,  # REQUIRED: This maps to Bedrock AgentCore actor_id under the hood
    }
}

## 단계 7: Agent 테스트

음식 선호도에 대한 대화를 통해 영양 어시스턴트를 테스트해 봅시다. Agent는 향후 사용을 위해 사용자 선호도를 자동으로 추출하고 저장합니다.

In [ ]:
# Helper function to pretty print agent output while running
def run_agent(query: str, config: RunnableConfig):
    printed_ids = set()  # 중복 출력 방지용 ID 세트
    events = graph.stream(
        {"messages": [{"role": "user", "content": query}]},
        config,
        stream_mode="values",
    )
    for event in events:
        if "messages" in event:
            for msg in event["messages"]:
                # Check if we've already printed this message
                if id(msg) not in printed_ids:
                    msg.pretty_print()
                    printed_ids.add(id(msg))


prompt = """
Hey there! Im cooking one of my favorite meals tonight, salmon with rice and veggies (healthy). Has
great macros for my weightlifting competition that is coming up. What can I add to this dish to make it taste better
and also improve the protein and vitamins I get?
"""

run_agent(prompt, config)

### 무엇이 저장되었나요?
보시다시피, model은 아직 우리의 선호도나 식이 제한 사항에 대한 통찰력이 없습니다.

pre/post model hooks를 사용한 이 구현에서는 두 개의 메시지가 여기에 저장되었습니다. 사용자의 첫 번째 메시지와 AI model의 응답이 모두 AgentCore Memory에 대화 이벤트로 저장되었습니다. 장기 메모리가 추출되는 데 몇 분 정도 걸릴 수 있으므로 첫 번째 시도에서 아무것도 발견되지 않으면 몇 초 후에 다시 시도하세요.

그런 다음 이러한 메시지는 fact 및 사용자 선호도 namespaces의 AgentCore 장기 메모리로 추출되었습니다. 실제로 store를 직접 확인하여 지금까지 저장된 내용을 확인할 수 있습니다:

In [ ]:
# Search our user preferences namespace
search_namespace = (actor_id, "preferences/")  # 사용자 선호도 namespace 지정
result = store.search(search_namespace, query="food", limit=3)  # "food" 키워드로 검색
print(f"Preferences namespace result: {result}")

### Store에 대한 Agent 액세스

**참고** - AgentCore 메모리는 백그라운드에서 이러한 이벤트를 처리하므로 메모리가 추출되고 장기 메모리 검색에 임베딩되는 데 몇 초가 걸릴 수 있습니다.

좋습니다! 이제 대화의 이전 메시지를 기반으로 장기 메모리가 namespaces로 추출된 것을 확인했습니다.

이제 새 세션을 시작하고 저녁 식사로 무엇을 요리할지에 대한 추천을 요청해 봅시다. Agent는 store를 사용하여 추출된 장기 메모리에 액세스하여 사용자가 확실히 좋아할 추천을 할 수 있습니다.

In [ ]:
config = {
    "configurable": {
        "thread_id": "session-2",  # New session ID
        "actor_id": actor_id,  # Same actor ID
    }
}

run_agent("Today's a new day, what should I make for dinner tonight?", config)

### 마무리

보시다시피, agent는 사용자 선호도 namespace 검색에서 pre-model hook 컨텍스트를 받았고 fact namespace에서 장기 메모리를 자체적으로 검색하여 사용자를 위한 포괄적인 답변을 만들 수 있었습니다.

AgentCoreMemoryStore는 매우 유연하며 pre/post model hooks 또는 store 작업이 있는 tools 자체를 포함하여 다양한 방식으로 구현할 수 있습니다. checkpointing을 위한 AgentCoreMemorySaver와 함께 사용하면 전체 대화 상태와 장기 통찰력을 결합하여 복잡하고 지능적인 agent 시스템을 구성할 수 있습니다.